In [10]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import mplfinance as mpf
import time
import yfinance as yf

import os
from os import listdir
from os.path import isfile, join

import statsmodels.api as sum
import seaborn as sns
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

In [11]:
#default Values
path = "D:\\Personal\\Quantitative-Finance\\stocks\\"
S_year = 2023
S_month = 1
S_day = 1
S_date_str = f"{S_year}-{S_month}-{S_day}"
S_date_datetime = dt.datetime(S_year,S_month,S_day)

E_year = 2024
E_month = 5
E_day = 31
E_date_str = f"{E_year}-{E_month}-{E_day}"
E_date_datetime = dt.datetime(E_year,E_month,E_day)


In [12]:
# # get stock file names in a list
files = [x for x in listdir(path) if isfile(join(path,x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers.sort()
# # create dataframe from our list
# stock_df = pd.DataFrame(tickers,columns=["tickers"])

# return dataframe from csv
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(path+ticker+".csv",index_col='Date',parse_dates=True)
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df
    

# Save dataframe to csv
def save_dataframe_to_csv(df,ticker):
    df.to_csv(path+ticker+".csv")

# return on investment over time
def get_roi(df):
    df['Date'] = pd.to_datetime(df['Date'])
    start_val = df[df['Date'] == S_date_str]['Adj Close'][0]
    end_val = df[df['Date'] == E_date_str]['Adj Close'][0]
    print("Initial Price:",start_val)
    print("Final Price:",end_val)
    roi = (end_val - start_val) / start_val
    return roi

# get coefficient of variance
def get_cov(stock_df):
    for stock in stock_df:
        mean,sd = stock_df['Adj Close'].mean(),stock_df['Adj Close'].std()
        cov = sd / mean
        return cov
    

# Merge Multiple stock in on df
def merge_df_by_column_name(col_name,*tickers):
    
    mult_df = pd.DataFrame()
    start = S_date_datetime
    end = E_date_datetime
    
    for x in tickers:
        mult_df[x] = get_df_from_csv(x)[col_name]
    return mult_df

def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending
    
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi
    
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

In [13]:
def calc_projected_roi(ticker):
    
    # Load and preprocess data
    a_df = get_df_from_csv(ticker)
    a_df = a_df.asfreq('d') # change frequency to day
    a_df = a_df.ffill()
    a_df.drop(a_df.columns[a_df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
    a_df = a_df.drop(['daily_return'], axis=1)


    # Determine optimal lags and fit model
    lags = ar_select_order(a_df, maxlag=30)
    model = AutoReg(a_df['Adj Close'], lags.ar_lags)
    model_fit = model.fit()

    # Split data into training and testing sets
    n = int(len(a_df) * 0.8)
    train_df = a_df.iloc[50:n]
    test_df = a_df.iloc[n:]

    # Fit the model on the training data
    train_model = AutoReg(a_df['Adj Close'], 600).fit(cov_type="HC0")
    start = len(train_df)
    end = len(train_df) + len(test_df) - 1

    # Make predictions
    prediction = train_model.predict(start=start, end=end, dynamic=True)

    # Forecast future values
    forecast = train_model.predict(start=end, end=end+160, dynamic=True)

    s_price = forecast.head(1).iloc[0]

    # Get the last price of prediction
    e_price = forecast.iloc[-1]

    # Get return over prediction
    return (e_price - s_price) / s_price

In [14]:
# calc_projected_roi('GOOG')

In [15]:
def get_proj_rois():
    ticker = []
    roi = []

    for x in tickers:
        print("working on : ",x)
        try:
            the_roi = calc_projected_roi(x)
        except Exception as ex:
            print("Stock Data Corrupted")
        else:
            ticker.append(x)
            print("ROI :",the_roi)
            roi.append(the_roi)
    return pd.DataFrame({'Ticker':ticker, 'ROI':roi})

In [17]:
proj_roi_df = get_proj_rois()
proj_roi_df

working on :  A
ROI : -0.1597419117044458
working on :  AAL
ROI : 0.07293559166681296
working on :  AAP
ROI : 0.4570777372921289
working on :  AAPL
ROI : 0.39348070635477506
working on :  ABBV
ROI : -0.09312191731555657
working on :  ABC
Stock Data Corrupted
working on :  ABG
ROI : 0.06603266782666428
working on :  ABMD
Stock Data Corrupted
working on :  ABT
ROI : 0.021553628908272685
working on :  ACN
ROI : -0.11952654344399859
working on :  ADBE
ROI : -0.07414838634068725
working on :  ADI
ROI : 0.15884889152043088
working on :  ADM
ROI : -0.3394961612047964
working on :  ADP
ROI : -0.09956986430602373
working on :  ADSK
ROI : -0.008115049379347141
working on :  AEE
ROI : -0.05125249972221844
working on :  AEP
ROI : 0.03237525369070257
working on :  AES
ROI : 0.2882753356123784
working on :  AFL
ROI : 0.23626340799850803
working on :  AIG
ROI : 0.40825638569237327
working on :  AIZ
ROI : -0.17288086266214517
working on :  AJG
ROI : 0.09431914352982476
working on :  AKAM
ROI : -0.9699

,Ticker,ROI
0,A,-0.159742
1,AAL,0.072936
2,AAP,0.457078
3,AAPL,0.393481
4,ABBV,-0.093122
...,...,...
468,YUM,0.005629
469,ZBH,-0.046248
470,ZBRA,0.342688
471,ZION,0.523566


In [18]:
proj_roi_df.sort_values(by=['ROI'], ascending=False)[:20]

,Ticker,ROI
263,LUMN,4.518432
317,NVDA,3.078127
227,ISRG,1.202278
109,CTLT,0.874382
180,GL,0.863267
45,AVGO,0.781005
257,LLY,0.700237
417,TSLA,0.628142
382,SJM,0.556680
419,TT,0.536808
